In [52]:
import pandas as pd
import numpy as np

In [53]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv'
!wget $data

--2021-09-26 20:13:11--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7077973 (6,8M) [text/plain]
Saving to: 'AB_NYC_2019.csv.1'

     0K .......... .......... .......... .......... ..........  0%  796K 9s
    50K .......... .......... .......... .......... ..........  1%  936K 8s
   100K .......... .......... .......... .......... ..........  2% 1,06M 7s
   150K .......... .......... .......... .......... ..........  2% 1,48M 7s
   200K .......... .......... .......... .......... ..........  3% 1,18M 6s
   250K .......... .......... .......... .......... ..........  4% 1,26M 6s
   300K .......... .......... .......... .......... ..........  5% 1,18M 6s
   350K .......... ...

In [54]:
df = pd.read_csv('AB_NYC_2019.csv')

  6800K .......... .......... .......... .......... .......... 99% 1,24M 0s
  6850K .......... .......... .......... .......... .......... 99%  968K 0s
  6900K .......... ..                                         100% 95,3M=5,8s

2021-09-26 20:13:17 (1,17 MB/s) - 'AB_NYC_2019.csv.1' saved [7077973/7077973]



In [55]:
features = ['neighbourhood_group',
'room_type',
'latitude',
'longitude',
'price',
'minimum_nights',
'number_of_reviews',
'reviews_per_month',
'calculated_host_listings_count',
'availability_365']

In [56]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

In [57]:
df[features].isnull().sum()

neighbourhood_group               0
room_type                         0
latitude                          0
longitude                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

### Question 1

In [58]:
df_new = df[features]
df_new['neighbourhood_group'].mode()

0    Manhattan
dtype: object

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
df_full_train, df_test = train_test_split(df_new, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [61]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [62]:
y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

In [64]:
del df_train['price']
del df_val['price']
del df_test['price']

### Question 2

In [73]:
numerical = ['latitude','longitude','minimum_nights','number_of_reviews',
     'reviews_per_month','calculated_host_listings_count','availability_365']

In [81]:
cm = df_train[numerical].corr()
cm

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [115]:
df_train['above_average'] = (y_train >= 152).astype(int)
df_train.head()

,neighbourhood_group,room_type,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,above_average
0,Brooklyn,Entire home/apt,40.72760,-73.94495,3,29,0.70,13,50,0
1,Manhattan,Private room,40.70847,-74.00498,1,0,0.00,1,7,0
2,Bronx,Entire home/apt,40.83149,-73.92766,40,0,0.00,1,0,0
3,Brooklyn,Entire home/apt,40.66448,-73.99407,2,3,0.08,1,0,0
4,Manhattan,Private room,40.74118,-74.00012,1,48,1.80,2,67,0


### Question 3

In [105]:
from  sklearn.metrics import mutual_info_score

In [106]:
categorical = ['neighbourhood_group','room_type']

In [119]:
def mutual_info_price_score(series):
    return mutual_info_score(series, df_train['above_average'])

In [121]:
round(df_train[categorical].apply(mutual_info_price_score),2)

neighbourhood_group    0.05
room_type              0.14
dtype: float64

### Question 4

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [129]:
dv = DictVectorizer(sparse = False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [145]:
y_train_bin = df_train['above_average'].values

In [147]:
model = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model.fit(X_train, y_train_bin)

LogisticRegression(random_state=42, solver='liblinear')

In [196]:
y_pred = model.predict_proba(X_val)[:, 1]

In [199]:
price_above_average = y_pred >= y_train_bin.mean()

In [200]:
y_val_bin = (y_val >= 152)

In [202]:
accuracy = (y_val_bin == price_above_average).mean()
accuracy.round(2)

0.75

### Question 5

In [226]:
small = ['neighbourhood_group', 'room_type', 'number_of_reviews', 'reviews_per_month']

In [208]:
dicts_train_small = df_train[small].to_dict(orient = 'records')
dicts_val_small = df_val[small].to_dict(orient = 'records')

In [209]:
dv_small = DictVectorizer(sparse = False)
dv_small.fit(dicts_train_small)

DictVectorizer(sparse=False)

In [212]:
X_train_small = dv_small.transform(dicts_train_small)

In [213]:
model_small = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model_small.fit(X_train_small, y_train_bin)

LogisticRegression(random_state=42, solver='liblinear')

In [219]:
X_val_small = dv_small.transform(dicts_val_small)
y_pred = model_small.predict_proba(X_val_small)[:, 1]

In [220]:
price_above_average = y_pred >= y_train_bin.mean()

In [236]:
original_accuracy = ((y_val >=152) == price_above_average).mean()
original_accuracy

0.7496676551794662

In [242]:
small1 = ['neighbourhood_group', 'room_type', 'number_of_reviews']

dicts_train_small1 = df_train[small1].to_dict(orient = 'records')
dicts_val_small1 = df_val[small1].to_dict(orient = 'records')

dv_small1 = DictVectorizer(sparse = False)

X_train_small1 = dv_small1.fit_transform(dicts_train_small1)
X_val_small1 = dv_small1.transform(dicts_val_small1)

model_small1 = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model_small1.fit(X_train_small1, y_train_bin)

y_pred = model_small1.predict_proba(X_val_small1)[:, 1]
price_above_average = y_pred >= y_train_bin.mean()
accuracy1 = ((y_val >=152) == price_above_average).mean()
diff = (original_accuracy - accuracy1).round(4)

print("The model accuracy without the feature reviews_per_month is: ", accuracy)
print("The accuracy difference is: ", diff)

The model accuracy without the feature reviews_per_month is:  0.7543716126393292
The accuracy difference is:  0.0


In [243]:
small1 = ['neighbourhood_group', 'room_type', 'reviews_per_month']

dicts_train_small1 = df_train[small1].to_dict(orient = 'records')
dicts_val_small1 = df_val[small1].to_dict(orient = 'records')

dv_small1 = DictVectorizer(sparse = False)

X_train_small1 = dv_small1.fit_transform(dicts_train_small1)
X_val_small1 = dv_small1.transform(dicts_val_small1)

model_small1 = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model_small1.fit(X_train_small1, y_train_bin)

y_pred = model_small1.predict_proba(X_val_small1)[:, 1]
price_above_average = y_pred >= y_train_bin.mean()
accuracy1 = ((y_val >=152) == price_above_average).mean()
diff = (original_accuracy - accuracy1).round(4)

print("The model accuracy without the feature number_of_reviews is: ", accuracy)
print("The accuracy difference is: ", diff)

The model accuracy without the feature number_of_reviews is:  0.7543716126393292
The accuracy difference is:  0.0009


In [244]:
small1 = ['neighbourhood_group', 'number_of_reviews', 'reviews_per_month']

dicts_train_small1 = df_train[small1].to_dict(orient = 'records')
dicts_val_small1 = df_val[small1].to_dict(orient = 'records')

dv_small1 = DictVectorizer(sparse = False)

X_train_small1 = dv_small1.fit_transform(dicts_train_small1)
X_val_small1 = dv_small1.transform(dicts_val_small1)

model_small1 = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model_small1.fit(X_train_small1, y_train_bin)

y_pred = model_small1.predict_proba(X_val_small1)[:, 1]
price_above_average = y_pred >= y_train_bin.mean()
accuracy1 = ((y_val >=152) == price_above_average).mean()
diff = (original_accuracy - accuracy1).round(4)

print("The model accuracy without the feature room_type is: ", accuracy)
print("The accuracy difference is: ", diff)

The model accuracy without the feature room_type is:  0.7543716126393292
The accuracy difference is:  0.0953


In [245]:
small1 = ['room_type', 'number_of_reviews', 'reviews_per_month']

dicts_train_small1 = df_train[small1].to_dict(orient = 'records')
dicts_val_small1 = df_val[small1].to_dict(orient = 'records')

dv_small1 = DictVectorizer(sparse = False)

X_train_small1 = dv_small1.fit_transform(dicts_train_small1)
X_val_small1 = dv_small1.transform(dicts_val_small1)

model_small1 = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
model_small1.fit(X_train_small1, y_train_bin)

y_pred = model_small1.predict_proba(X_val_small1)[:, 1]
price_above_average = y_pred >= y_train_bin.mean()
accuracy1 = ((y_val >=152) == price_above_average).mean()
diff = (original_accuracy - accuracy1).round(4)

print("The model accuracy without the feature neighbourhood_group is: ", accuracy)
print("The accuracy difference is: ", diff)

The model accuracy without the feature neighbourhood_group is:  0.7543716126393292
The accuracy difference is:  0.0267


### Question 6

In [247]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

In [253]:
df.price = df.price.apply(np.log1p)

In [265]:
X = df.drop(['price','id','name','host_id', 'host_name','neighbourhood', 'above_average'], axis=1)
y = df.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [266]:
ridge_model = Ridge(alpha=0)
ridge_model.fit(X_train.values.reshape(-1,1), y_train)

ValueError: could not convert string to float: 'Brooklyn'